In [38]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import importlib
import numpy as np
from scipy.linalg import eigh

%matplotlib ipympl
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
# np.set_printoptions(linewidth=200, formatter={'float': '{:.3e}'.format})
np.set_printoptions(linewidth=200, formatter={'float': '{:.6f}'.format})

In [39]:
# Fundamental constants
h    = 6.62607015e-34
GHz  = 1e9
fF = 1e-15
nH = 1e-9
e0   = 1.602176634e-19
Φ_0 = h/(2*e0)

In [40]:
# Circuit parameters
Csh = 15
C   = 15
Cg  = 10
CJ  = 3
Lq  = 25
Lr  = 10
Δ   = 0.1
EJ  = 10.0
l = Lq*(Lq+4*Lr) - 4*Δ**2

In [41]:
Cf = C/2 + Csh + CJ
Lf = l/(Lq+4*Lr)
C_f = Cf * 1e-15
L_f = Lf * 1e-9
Z_f = np.sqrt(L_f/C_f)

L_r = l/Lq * 1e-9
C_r = C/2 * 1e-15
Z_r = np.sqrt(L_r/C_r)

In [42]:
qubit = sq_ext.KIT_qubit()
fluxonium = sq_ext.KIT_fluxonium()
resonator = sq_ext.KIT_resonator()

In [43]:
fluxonium.diag(2)
resonator.diag(2);

In [50]:
# Parameters
ω_f = fluxonium.efreqs[1]-fluxonium.efreqs[0]  # Fluxonium frequency
ω_r = resonator.efreqs[1]-resonator.efreqs[0]  # Resonator frequency

Φ_f = fluxonium.flux_op(0)
Φ_r = resonator.flux_op(0)
g_Φ = 2 * Δ / (l * nH) * Φ_f[0,1] * Φ_r[0,1] / 2 / np.pi / GHz

# Pauli matrices for the fluxonium
sigma_x = np.array([[0, 1], [1, 0]])
sigma_z = np.array([[1, 0], [0, -1]])

sigma_p = np.array([[0, 1], [0, 0]])
sigma_m = np.array([[0, 0], [1, 0]])

# Resonator operators
def create(n):
    return np.diag(np.sqrt(np.arange(1, n)), -1)

def annihilate(n):
    return np.diag(np.sqrt(np.arange(1, n)), 1)

# Number of states for the resonator (0, 1, 2)
n = 3

# Creation and annihilation operators for the resonator
a = create(n)
a_dagger = annihilate(n)

# Identity matrices
I_resonator = np.eye(n)
I_fluxonium = np.eye(2)

# Isolated Hamiltonians
H_fluxonium = ω_f / 2 * sigma_z
H_resonator = ω_r * np.dot(a_dagger, a)

# Construct the system Hamiltonian with tensor products
H_system = np.kron(H_resonator, I_fluxonium) + np.kron(I_resonator, H_fluxonium)
H_interaction = g_Φ * np.kron(a + a_dagger, sigma_x)
# H_interaction = g_Φ * (np.kron(a_dagger, sigma_p)+np.kron(a, sigma_m)) # Applying the RWA

# Total Hamiltonian
H_total = H_system + H_interaction

In [51]:
# Find the eigenvalues (energy levels)
energies = eigh(H_total, eigvals_only=True)

energies-energies[0]

array([0.000000, 1.945486, 7.208392, 9.153866, 14.416909, 16.362372])

In [49]:
# Find the eigenvalues (energy levels)
energies = eigh(H_total, eigvals_only=True)

energies-energies[0]

array([0.000000, 1.945531, 7.208422, 9.153866, 14.416864, 16.362342])

In [47]:
qubit.diag(8)
qubit.efreqs-qubit.efreqs[0]

array([0.000000, 1.945361, 6.161082, 7.208451, 9.153874, 10.792941, 13.369603, 14.416902])